In [6]:
import torch
import torch.nn as nn
import numpy as np

# Create simple sequence data (sine wave prediction)
def create_sequence_data(seq_length=10, num_samples=1000):
    # Generate sine wave data
    x = np.linspace(0, 4*np.pi, num_samples)
    y = np.sin(x)
    
    # Create sequences: use seq_length points to predict next point
    X, Y = [], []
    for i in range(len(y) - seq_length):
        X.append(y[i:i + seq_length])
        Y.append(y[i + seq_length])
    
    return np.array(X), np.array(Y)

# Generate data
seq_length = 10
X, Y = create_sequence_data(seq_length)

# Convert to PyTorch tensors
X = torch.FloatTensor(X).unsqueeze(-1)  # Add feature dimension
Y = torch.FloatTensor(Y).unsqueeze(-1)

print(f"Input shape: {X.shape}")  # [samples, sequence_length, features]
print(f"Output shape: {Y.shape}")

# Create LSTM model components directly
input_size = 1
hidden_size = 50
output_size = 1

# LSTM layer
lstm_layer = nn.LSTM(input_size, hidden_size, batch_first=True)

# Linear layer for final prediction
linear_layer = nn.Linear(hidden_size, output_size)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(list(lstm_layer.parameters()) + list(linear_layer.parameters()), lr=0.001)

# Training function
def train_step(X, Y):
    # Forward pass
    lstm_output, (hidden, cell) = lstm_layer(X)
    
    # Use last time step output for prediction
    last_output = lstm_output[:, -1, :]
    predictions = linear_layer(last_output)
    
    # Calculate loss
    loss = criterion(predictions, Y)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()

# Training loop
print("\nTraining LSTM...")
for epoch in range(100):
    loss = train_step(X, Y)
    
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.6f}')

# Prediction function
def predict(input_sequence):
    with torch.no_grad():
        lstm_output, (hidden, cell) = lstm_layer(input_sequence)
        last_output = lstm_output[:, -1, :]
        prediction = linear_layer(last_output)
        return prediction

# Test the model
print("\nTesting LSTM...")
test_input = X[:1]  # First sequence
print(f"Test input (first 10 points): {test_input.squeeze().numpy()}")

# Generate multiple predictions
predictions = []
current_input = test_input

for i in range(5):
    pred = predict(current_input)
    predictions.append(pred.item())
    
    # Update input: remove first point, add prediction
    new_input = torch.cat([current_input[:, 1:, :], pred.unsqueeze(1)], dim=1)
    current_input = new_input

print(f"LSTM predictions (next 5 points): {predictions}")
print(f"Actual values: {Y[1:6].squeeze().numpy()}")

# Count total parameters
total_params = sum(p.numel() for p in lstm_layer.parameters()) + sum(p.numel() for p in linear_layer.parameters())
print(f"\nTotal parameters: {total_params}")

Input shape: torch.Size([990, 10, 1])
Output shape: torch.Size([990, 1])

Training LSTM...
Epoch 0, Loss: 0.483960
Epoch 20, Loss: 0.292002
Epoch 40, Loss: 0.027867
Epoch 60, Loss: 0.010110
Epoch 80, Loss: 0.004901

Testing LSTM...
Test input (first 10 points): [0.         0.01257862 0.02515525 0.03772789 0.05029457 0.06285329
 0.07540207 0.0879389  0.10046184 0.11296887]
LSTM predictions (next 5 points): [0.06871828436851501, 0.07810980081558228, 0.08587592840194702, 0.09258948266506195, 0.09847068786621094]
Actual values: [0.13792734 0.15037481 0.16279851 0.17519644 0.18756665]

Total parameters: 10651
